In [ ]:
import pymysql

def select_from_mysql(id_min, id_max):
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='tuhu_maintenance', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE tuhu_maintenance")
    sql = '''
        SELECT t1.tid,tuhu_bytype.id,t1.Pid
        FROM
            (SELECT DISTINCT tid,BaoYangType,PackageType,Pid 
            FROM `tuhu_sales_vehicle_bytype_and_pid`
            WHERE id BETWEEN %s AND %s) AS t1
        LEFT JOIN `tuhu_bytype`
        ON t1.BaoYangType = tuhu_bytype.BaoYangType AND t1.PackageType = tuhu_bytype.PackageType
    ''' % (id_min, id_max)
    cur.execute(sql)
    result =cur.fetchall()
    cur.close()
    conn.close()
    return result

def insert_into_mysql(lists):
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='tuhu_maintenance', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE tuhu_maintenance")
    for ii in lists:
        sql = '''
            INSERT INTO `tuhu_simple_result`
            (tid,bytype_id,pid)
            VALUES
            {}
        '''.format(ii)
        cur.execute(sql)
        
    conn.commit()
    cur.close()
    conn.close()
    

# 分批查询存入（10000条一次）

for ii in range(0, 3015):
    id_min = (10000 * ii) + 1
    id_max = (1 + ii) * 10000
    tuples = select_from_mysql(id_min, id_max)
    insert_into_mysql(tuples)
    print('第%s次存入成功(id<%s)' % (ii+1, id_max))
    

    

In [ ]:
import pymysql

def select_from_mysql():
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='tuhu_maintenance', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE tuhu_maintenance")
    sql = '''
        SELECT tid,bytype_id,pid
        FROM `tuhu_simple_result`
        GROUP BY tid,bytype_id,pid
    '''
    cur.execute(sql)
    result =cur.fetchall()
    cur.close()
    conn.close()
    return result

def insert_into_mysql(tuples):
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='tuhu_maintenance', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE tuhu_maintenance")
    for ii in tuples:
        sql = '''
            INSERT INTO `tuhu_simple_result_copy1`
            (tid,bytype_id,pid)
            VALUES
            {}
        '''.format(ii)
        cur.execute(sql)
        
    conn.commit()
    cur.close()
    conn.close()
    

# 分批存入（10000条一次）
print('正在查询去重。。')
tuples = select_from_mysql()
print(len(tuples))
print('正在插入新表。。。。')
for j in range(0, int(len(tuples)/10000)+1):
    id_min = j * 10000
    id_max = id_min + 10000
    one_tuple = tuples[id_min:id_max]
    insert_into_mysql(one_tuple)
    print('第%s次(%s)存入成功' % (j+1,int(len(tuples)/10000)))
print('GAME OVER!')
    

    












In [40]:
aa = [
'40',
'1H2',
'2PU',
'2SZ',
'1F8',
'8S6',
'9AE',
'8S7',
'1D0',
'8B6',
'586',
'8S9',
'3L8',
'580',
'8R3',
'776',
'9AC',
'3P0',
'4R0',
'5A6',
'4Q2',
'2QJ',
'2QL',
'2NJ',
'2QM',
'4U1',
'2QK',
'2QH',
'D6S',
'1K0',
'2PN',
'2KR',
'2QG',
'2NE',
'2ND',
'791'
]
print(aa)

['40', '1H2', '2PU', '2SZ', '1F8', '8S6', '9AE', '8S7', '1D0', '8B6', '586', '8S9', '3L8', '580', '8R3', '776', '9AC', '3P0', '4R0', '5A6', '4Q2', '2QJ', '2QL', '2NJ', '2QM', '4U1', '2QK', '2QH', 'D6S', '1K0', '2PN', '2KR', '2QG', '2NE', '2ND', '791']


In [39]:
def be_really(vins, vin_head, year):
    vin_end = vins.split("-")[1]
    v = get_right_vin(vin_head, vin_end[0], int(vin_end[1:]), year)
    return v

def get_right_vin(vin8, vin2, vin_end, year):
    """
    根据权重校验获取有效的vin(该vin并不一定存在)
    :param vin8: vin的前8位
    :param vin2: vin的10,11位
    :param vin_end: vin的后6位
    :return: 有效的vin
    """
    # vin中字母对应的值
    dict_alp = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'J': 1, 'K': 2, 'L': 3, 'M': 4,
                'N': 5, 'P': 7, 'R': 9, 'S': 2, 'T': 3, 'U': 4, 'V': 5, 'W': 6, 'X': 7, 'Y': 8, 'Z': 9,
                '0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9, '*': 0}
    # vin中数字对应的加权系数值，第9位是权重位
    dict_index = {1: 8, 2: 7, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 10, 9: 0,
                  10: 9, 11: 8, 12: 7, 13: 6, 14: 5, 15: 4, 16: 3, 17: 2}
    # 年份表
    dict_year = {
        2020: 'L', 2019: 'K', 2018: 'J', 2017: 'H', 2016: 'G', 2015: 'F', 2014: 'E', 2013: 'D', 2012: 'C',
        2011: 'B', 2010: 'A', 2009: '9', 2008: '8', 2007: '7', 2006: '6', 2005: '5', 2004: '4', 2003: '3'
    }
    vin2 = dict_year[int(year)] + vin2
    # 前八位权重值
    q1 = 0
    for i in range(1, len(vin8) + 1):
        q1 += (dict_alp[vin8[i - 1]] * dict_index[i])

    # 第9位暂时补‘*’，待算出v9后替换掉‘*’
    vin = vin8 + "*" + vin2 + "%06d" % vin_end
    # 权重值
    q2 = 0
    for i in range(10, len(vin) + 1):
        q2 += (dict_alp[vin[i - 1]] * dict_index[i])
    # 确认vin的第9位的值
    v9 = "X" if (q1 + q2) % 11 == 10 else str((q1 + q2) % 11)
    vin = vin.replace("*", v9)
    return vin

vins = 'GDJ150-K000116'
vin_head = 'JTEBR3FJ'
year = '2015'
vin = be_really(vins, vin_head, year)
print(vin)

JTEBR3FJ5FK000116
